In [2]:
import numpy as np
import random
from implementations import *
from helpers import load_csv_data


In [2]:
seed = 7

In [18]:
train_labels_raw, train_data_raw, train_ids = load_csv_data("data/train.csv")
_, test_data_raw, test_ids = load_csv_data("data/train.csv")


In [5]:
import pandas as pd

df = pd.DataFrame(data=train_data_raw, index=None, columns=None)
df = df.applymap(lambda v: v if v > -999.0 else np.NaN)
print(len(df))
two_more_jets = df[(df[[22]] >= 2).to_numpy()].dropna()
one_jet = df[(df[[22]] == 1).to_numpy()].drop([4,5,6,12, 26,27,28], axis=1).dropna()
zero_jet = df[(df[[22]] == 0).to_numpy()].drop([4,5,6,12,23,24,25,26,27,28], axis=1).dropna()
print(len(two_more_jets)+len(one_jet)+len(zero_jet))
df.describe()

250000
211886


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,211886.000000,250000.000000,250000.000000,250000.000000,72543.000000,72543.000000,72543.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,150087.000000,150087.000000,150087.000000,72543.000000,72543.000000,72543.000000,250000.000000
mean,121.858528,49.239819,81.181982,57.895962,2.403735,371.783360,-0.821688,2.373100,18.917332,158.432217,...,-0.010119,209.797178,0.979176,84.822105,-0.003275,-0.012393,57.679474,-0.011845,-0.001582,73.064591
std,57.298157,35.344886,40.828691,63.655682,1.742226,397.699325,3.584362,0.782911,22.273494,115.706115,...,1.812223,126.499506,0.977426,60.662276,1.784546,1.813385,31.985782,2.031743,1.816950,98.015662
min,9.044000,0.000000,6.329000,0.000000,0.000000,13.602000,-18.066000,0.208000,0.000000,46.104000,...,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,30.000000,-4.500000,-3.142000,0.000000
25%,91.885250,19.241000,59.388750,14.068750,0.882500,111.977000,-2.629000,1.810000,2.841000,77.550000,...,-1.575000,123.017500,0.000000,44.422500,-1.342000,-1.584000,37.312000,-1.612000,-1.576500,0.000000
50%,112.406000,46.524000,73.752000,38.467500,2.107000,225.885000,-0.244000,2.491500,12.315500,120.664500,...,-0.024000,179.739000,1.000000,65.561000,0.000000,-0.033000,47.902000,-0.010000,-0.002000,40.512500
75%,135.482000,73.598000,92.259000,79.169000,3.690000,478.226000,0.958000,2.961000,27.591000,200.478250,...,1.561000,263.379250,2.000000,103.342000,1.336000,1.562000,66.637000,1.589500,1.576000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [19]:
med_over = train_data_raw[train_data_raw[:,0] != -999]
med_DER_mass_MMC = np.median(med_over[:,0])

In [20]:
train_data = train_data_raw[train_data_raw[:,0] > -999].copy()
train_labels = train_labels_raw[train_data_raw[:,0] > -999].copy()
test_data = test_data_raw.copy()
test_data[test_data_raw[:,0] == -999] = med_DER_mass_MMC

In [21]:
two_more_jets = train_data[train_data[:,22] >= 2].copy()
one_jet = train_data[train_data[:,22] == 1].copy()
one_jet = np.delete(one_jet, np.s_[4,5,6,12, 26,27,28], axis=1)
zero_jet = train_data[train_data[:,22] == 0].copy()
zero_jet = np.delete(zero_jet, np.s_[4,5,6,12,23,24,25,26,27,28], axis=1)

jet_sets = [zero_jet, one_jet, two_more_jets]

In [22]:
two_more_jets = train_labels[train_data[:,22] >= 2].copy()
one_jet = train_labels[train_data[:,22] == 1].copy()
zero_jet = train_labels[train_data[:,22] == 0].copy()

jet_sets_labels = [zero_jet, one_jet, two_more_jets]

In [23]:
two_more_jets_test = test_data[test_data[:,22] >= 2].copy()
one_jet_test = test_data[test_data[:,22] == 1].copy()
one_jet_test = np.delete(one_jet_test, np.s_[4,5,6,12, 26,27,28], axis=1)
zero_jet_test = test_data[test_data[:,22] == 0].copy()
zero_jet_test = np.delete(zero_jet_test, np.s_[4,5,6,12,23,24,25,26,27,28], axis=1)


jet_sets_test = [zero_jet_test, one_jet_test, two_more_jets_test]

In [24]:
for i in range(3):
    mean = jet_sets[i].mean(axis=0)
    std = jet_sets[i].std(axis=0)
    jet_sets[i] = (jet_sets[i] - mean)/std
    jet_sets_test[i] = (jet_sets_test[i] - mean)/std

functions to test different methods for regression

In [1]:
def compute_loss(y, tx, model):
    pred = tx @ model
    decision = 0
    pred[np.where(pred <  decision)] = -1
    pred[np.where(pred >= decision)] = 1
    err = (pred - y)/2
    return .mean()

def build_validation_sets(y, k_fold, seed):
    """
    returns indices for a train set and a test set
    """
    num_row = y.shape[0]
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    return indices[indices.shape[0]//(k_fold+1):], indices[:indices.shape[0]//(k_fold+1)],

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree.
    
    Args:
        x: numpy array of shape (N,D)
        degree: integer.
        
    Returns:
        poly: numpy array of shape (N,D*d+1)
    """    
    N = x.shape[0]
    D = x.shape[1]
    #poly_base = np.zeros((N, D*degree + 1))
    poly_base = np.ones((N,1))
    poly_base = np.hstack((poly_base, x.copy()))
    for i in range(degree-1):
        range_start = 1 + i*D
        range_stop = 1 + (i+1)*D
        next_power = poly_base[:,range_start:range_stop]*x
        poly_base = np.hstack((poly_base, next_power))
    
    return poly_base

def cross_validation(train_model, degree):
    """
    Tests a certain training function using 4-fold cross-validation
    arguments: 
    train_model: func(y, tx) -> model
    returns: model, training_loss, test_loss
    """
    tx = build_poly(x, degree)
    k = 4
    train_idx, test_idx = build_validation_sets(x, k, seed)
    model, train_loss = train_model(y[train_idx], tx[train_idx])
    test_loss = compute_loss(y[test_idx], tx[test_idx], model)
    return model, train_loss, test_loss

Graph stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def cross_validation_visualization(lambds, mse_tr, mse_te):
    """visualization the curves of mse_tr and mse_te."""
    plt.semilogx(lambds, mse_tr, marker=".", color='b', label='train error')
    plt.semilogx(lambds, mse_te, marker=".", color='r', label='test error')
    plt.xlabel("lambda")
    plt.ylabel("r mse")
    #plt.xlim(1e-4, 1)
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    
def cross_validation_explore_lambda(train_model, degree, lambdas):
    """cross validation over regularisation parameter lambda.
    
    Args:
        train_model: func(y, tx, lambda) -> model
        degree: integer, degree of the polynomial expansion
        lambdas: shape = (p, ) where p is the number of values of lambda to test
    Returns:
        best_lambda : scalar, value of the best lambda
        best_mse : scalar, the associated mean squared error for the best lambda
    """
    seed = 12
    degree = degree
    lambdas = lambdas
    # define lists to store the loss of training data and test data
    mse_tr = []
    mse_te = []
    
    best_idx = 0
    idx = 0
    for lambda_ in np.nditer(lambdas):
        _, te, tr = cross_validation(lambda y,tx: train_model(y,tx, lambda_), degree)
        mse_tr.append(tr)
        mse_te.append(te)
        if te < mse_te[best_idx]:
            best_idx = idx
        idx += 1
    
    best_lambda = lambdas[best_idx]
    best_mse = mse_te[best_idx]
        
    cross_validation_visualization(lambdas, mse_tr, mse_te)
    print("For polynomial expansion up to degree %.f, the choice of lambda which leads to the best test mse is %.5f with a test mse of %.3f" % (degree, best_lambda, best_mse))
    return best_lambda, best_mse

lambdas = np.logspace(-4, 0, 30)
#best_lambda, best_rmse = cross_validation_demo(7, 4, np.logspace(-4, 0, 30))

Now try different methods

In [ ]:
cross_validation(least_squares,2)

In [ ]:
cross_validation_explore_lambda(lambda y,tx,lambda_: ridge_regression(y,tx,lambda_), 3, lambdas)

In [ ]:
cross_validation(lambda y,tx: logistic_regression(y, tx, np.zeros(tx.shape[1]), 100, 0.001), 1)